# Import Required Libraries
Import necessary libraries such as requests, pandas, and folium.

In [1]:
# Import Required Libraries
import requests  # For making HTTP requests to the Overpass API
import pandas as pd  # For data manipulation and analysis
import folium  # For creating interactive maps

# Fetch Data from OpenStreetMap using Overpass API
Use the Overpass API to fetch data on petrol stations in Switzerland.

In [2]:
# Fetch Data from OpenStreetMap using Overpass API

# Define the Overpass API endpoint
overpass_url = "http://overpass-api.de/api/interpreter"

# Define the Overpass query to fetch petrol stations in Switzerland
overpass_query = """
[out:json];
area["ISO3166-1"="CH"][admin_level=2];
node["amenity"="fuel"](area);
out body;
>;
out skel qt;
"""

# Make the request to the Overpass API
response = requests.get(overpass_url, params={'data': overpass_query})

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Error fetching data from Overpass API: {response.status_code}")

# Extract relevant information from the response
elements = data['elements']
stations = []
for element in elements:
    if 'tags' in element and 'name' in element['tags']:
        stations.append({
            'id': element['id'],
            'lat': element['lat'],
            'lon': element['lon'],
            'name': element['tags']['name']
        })

# Convert the list of stations to a DataFrame
stations_df = pd.DataFrame(stations)

# Display the first few rows of the DataFrame
stations_df.head()

,id,lat,lon,name
0,5702030,46.680546,7.827412,Migrol
1,5702071,46.683415,7.844559,Avia
2,17407058,47.054727,7.625379,Moveri Burgdorf
3,27069561,47.396391,8.623317,SOCAR Zur Schmiede
4,27125026,47.401093,8.619955,BP Garage Tresch


# Save Data in JSON and CSV Format
Save the fetched data in both JSON and CSV formats on the local computer.

In [3]:
# Save Data in JSON and CSV Format

# Save the data to a JSON file
stations_df.to_json('/workspaces/scientific_programming/Week_03/challenge/petrol_stations.json', orient='records', lines=True)

# Save the data to a CSV file
stations_df.to_csv('/workspaces/scientific_programming/Week_03/challenge/petrol_stations.csv', index=False)

# Count Petrol Stations per Operator
Use pandas' pivot_table() method to count the number of petrol stations per operator.

In [4]:
# Count Petrol Stations per Operator

# Use pandas' pivot_table() method to count the number of petrol stations per operator
pivot_table = stations_df.pivot_table(index='name', aggfunc='size')

# Sort the pivot table to identify the 5 largest operators
largest_operators = pivot_table.sort_values(ascending=False).head(5)

# Display the pivot table
largest_operators

name
Avia      172
Agrola    160
BP        115
Tamoil    103
Shell      78
dtype: int64

# Identify the 5 Largest Operators
Sort the pivot table to identify the 5 largest operators in terms of the total number of petrol stations.

In [5]:
# Sort the pivot table to identify the 5 largest operators
largest_operators = pivot_table.sort_values(ascending=False).head(5)

# Display the 5 largest operators
largest_operators

name
Avia      172
Agrola    160
BP        115
Tamoil    103
Shell      78
dtype: int64

# Create a Map to Visualize Petrol Stations
Create a map using folium to visualize all petrol stations in Switzerland, with popups showing the brand of each station.

In [6]:
# Create a Map to Visualize Petrol Stations

# Initialize a folium map centered around Switzerland
map_switzerland = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Add petrol stations to the map
for _, station in stations_df.iterrows():
    folium.Marker(
        location=[station['lat'], station['lon']],
        popup=station['name']
    ).add_to(map_switzerland)

# Save the map to an HTML file
map_switzerland.save('/workspaces/scientific_programming/Week_03/challenge/petrol_stations_map.html')

# Display the map
map_switzerland